![Banner](img/AI_Special_Program_Banner.jpg)

## Recurrent Neural Networks (RNN) - Exercise 1: Tiny Language Model for Limericks
---
Instructions are given in <span style="color:blue">blue</span> color.
(Unfortunately, Google Colab won't display the blue color.)

In this exercise, we want to deal with a common requirement when working with deep learning models: Models should be reusable.

As you may have noticed, depending on the complexity of the problem and the amount of data, the models' training can take quite some time (when done on a CPU). Therefore, the ability to save and load already trained models is crucial to develop efficient problem solutions.

Additionally, we want to make use of the possibility to run notebooks in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb). The inherent advantage - given over a local execution - is that Colab allows the use of GPUs and, consequently, training time should significantly speed up.

**Note**: For this exercise, you will need a Google account. If you don't already have one, please create one using [this website](https://accounts.google.com/signup/v2/webcreateaccount?hl=en&flowName=GlifWebSignIn&flowEntry=SignUp). If you are not comfortable in providing your personal information, don't do so. You can delete your account and associated data after completing the exercise by following [these](https://support.google.com/accounts/answer/32046?hl=en) instructions.

As for the actual contents of this week's exercise, we will look at character-level language modeling with RNNs. In particular, we are trying to train a network capable of producing text that (hopefully) resembles the structure of so-called [Limericks](https://en.wikipedia.org/wiki/Limerick_(poetry)).

> Deep Thought, after millions of years,
<br>
> With the ultimate answer appears,
<br>
> &nbsp;&nbsp;&nbsp;&nbsp;Which is just 42.
<br>
> &nbsp;&nbsp;&nbsp;&nbsp;This is certainly true,
<br>
> Though it sounds a bit strange to the ears.

<div style="text-align: right"><a href="http://www.oedilf.com/db/Lim.php?Word=42">Source</a></div>

---

## Overview
- [1. Setting up Google Colab](#1.-Setting-up-Google-Colab)
- [2. Data Preparation](#2.-Data-Preparation)
- [3. RNN Model Creation](#3.-RNN-Model-Creation)

---

## 1. Setting up Google Colab
---

* <div style="color:blue">Upload this notebook to Google Colab.</div>

    1. Visit https://colab.research.google.com/notebooks/intro.ipynb.
    2. Log in to your Google account.
    3. Select **`File`** **`Upload notebook`** from the menu bar.
    4. Upload this notebook.

* <div style="color:blue">Make sure to utilize GPUs.</div>

    1. Select **`Runtime`** **`Change Runtime type`** from the menu bar.
    2. Choose *T4 GPU* for hardware acceleration (if that is not already the case; otherwise just cancel the operation)

In [1]:
# ensure torchtext installed
try:
    import torchtext
except:
    print("Installing torchtext")
    !pip install torchtext   

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchtext
from torchtext.data import get_tokenizer
from torch import nn
from tqdm.notebook import trange, tqdm

import datetime as dt

# force gpu computing, when gpu library is available
USE_GPU = True


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
torch.manual_seed(42)

In [ ]:
torch.__version__

In [ ]:
CUDA_AVAILABLE = torch.cuda.is_available()
print(f'CUDA_AVAILABLE = {CUDA_AVAILABLE}')

The data we will work with is provided as simple continuous text. The limericks contained therein have been scraped from [this website](http://www.oedilf.com/db/Lim.php). You will find the text file `limericks.txt` in the `data` folder next to this notebook.

* <div style="color:blue">Upload the text file to Google Colab.</div>

    1. On the left-hand side of the Google Colab screen, there is a `Files` tab - click to open.
    2. You need to be connected to a runtime to see the session storage and run at least one code cell if not already connected.
    3. If connected, you can select to upload files to the session storage. Note that the session storage will be purged when the session is terminated.
    4. Upload `limericks.txt` to the session storage (this might take a while; to check whether the file has been uploaded, close and re-open the files tab).
    5. Run the following code cells to read the data.

In [ ]:
# Reading data
limericks_file = "limericks.txt"
limericks = open(limericks_file, 'r').read()

In [ ]:
# Looking at some limericks
print(limericks[9990:10790])

## 2. Data Preparation
---

In [ ]:
# Getting the unique characters used in the text
vocab = sorted(set(limericks))
print(vocab)
len(vocab)

In [ ]:
# Creating unique index for each character in the text
char_to_index = {char:index for index, char in enumerate(vocab)}
char_to_index

In [ ]:
# Checking index of a specific character
char_to_index['t']

In [ ]:
# Mapping the index back to the characters
index_to_char = np.array(vocab)
index_to_char

In [ ]:
# Checking character for a given index
index_to_char[59]

In [ ]:
# Encoding the entire text
encoded_text = np.array([char_to_index[char] for char in limericks])
encoded_text

* <div style="color:blue">How many characters are there in the entire text?</div>

In [ ]:
# Your solution goes here:

In [ ]:
# Comparing raw text ...
limericks[:100]

In [ ]:
# ... with encoded text
encoded_text[:100]

The next thing we have to take care of is to split the data into batches. For this, it is important to determine a sequence length for each batch (number of characters) that 

* is long enough to capture the general structure of a limerick, but
* is not too long, so that past information does not simply become noise.

* <div style="color:blue">Take a look at the limericks in <code>limericks.txt</code> and decide on a suitable <b>sequence length</b>.</div>

In [ ]:
sequence_length = 0 # <- your solution goes here

In [ ]:
class CharSequenceDataset(Dataset):

    def __init__(self, text_encoded, seq_length, offset):
        sequences = []
        target = []
        for i in range(0, len(text_encoded) - seq_length - 1, offset):
            seq = text_encoded[i: i + seq_length]
            sequences.append(seq)
            target.append(text_encoded[i + seq_length])
        self.sequences = torch.tensor(sequences)
        self.target = torch.tensor(target)
        
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.target[idx]
    

offset = sequence_length 

ds = CharSequenceDataset(encoded_text, sequence_length, offset)

## inspection:
for seq, target in ds:
    print(seq.numpy(), ' -> ', target.numpy())
    print(repr(''.join(index_to_char[seq])), 
          ' -> ', repr(''.join(index_to_char[target])))
    break

In [ ]:
# final preparation step: create a data loader
BATCH_SIZE = 128

trainloader = DataLoader(ds, shuffle=True, batch_size=BATCH_SIZE)
len(ds)

## 3. RNN Model Creation
---

In [ ]:
# Vocabulary length
vocab_size = len(vocab)
# Embedding dimensions
embed_dim = 64
# Number of RNN units (neurons)
rnn_units = 512

In [ ]:
class Network(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_units, recurrent_type='LSTM'):
        super(Network, self).__init__()
        self.recurrent_type = recurrent_type
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if recurrent_type == 'RNN':
            self.rnn = nn.RNN(embedding_dim, rnn_units, batch_first=True)
        elif recurrent_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, rnn_units, batch_first=True)
        elif recurrent_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, rnn_units, batch_first=True)
        self.linear = nn.Linear(rnn_units, vocab_size)
    
    def forward(self, x):
        embedded = self.embedding(x)
        #lstm also returns the cell hidden state
        if self.recurrent_type == "LSTM":
            output, (hidden, cell_hidden) = self.rnn(embedded)
        else:
            output, hidden = self.rnn(embedded)
        output = self.linear(hidden[-1])
        return output
    
# Instantiate the PyTorch model (and remember the type for saving the model ...)
rnn_type = 'LSTM'
model = Network(vocab_size, embed_dim, rnn_units, recurrent_type=rnn_type)
model

In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())

Setup the training loop

In [ ]:
def train(model, dataloader, optimizer, criterion):
    model.train() 
    
    running_loss = 0.0
    
    # Iterate over the training dataset
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device).long()
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        # Compute the loss
        loss = criterion(outputs, labels)
        # Backward pass
        loss.backward()
        # Update the weights
        optimizer.step()
        
        running_loss += loss.item()

    # Calculate the average loss for the epoch
    epoch_loss = running_loss / len(dataloader)
    return epoch_loss

def train_for_epochs(epchs, model, trainloader, optimizer, criterion):
    history = {
        "loss": []
    }
    loop = trange(epchs)
    # Training loop
    for epoch in loop:

        epoch_loss_train = train(model, trainloader, optimizer, criterion)   

        history["loss"].append(epoch_loss_train)

        loop.set_description(f"Train Loss: {epoch_loss_train:.4f}")
    return history

# record the total number of epochs for training so far (may be used for naming models ...)
tot_epochs = 0

* <div style="color:blue">Train the model for <b>2</b> epochs. Double-check that you enabled a GPU runtime (unless you don't mind waiting). However, even if you are using GPUs, it may take a while to complete the training.</div>

In [ ]:
# Your solution goes here:

* <div style="color:blue">Save the state dict of your model to your session storage.</div>

**Hint**: Have a look at the [documentation](https://pytorch.org/tutorials/beginner/saving_loading_models.html) to save and load models.

In [ ]:
# Your solution goes here:

* <div style="color:blue">Continue training for another <b>18</b> epochs. In total you should have trained the model for <b>20</b> epochs.</div>

In [ ]:
# Your solution goes here:

* <div style="color:blue">Save the state of of your model to your session storage, but use a different file name this time.</div>

In [ ]:
# Your solution goes here:

* <div style="color:blue">Download both saved models from your session storage to your local hard drive (place them in the <code>models</code> folder).</div>

**CAUTION**: If you accidentally close the current session without downloading the models first, those will be purged and your training progress is lost.

* <div style="color:blue">Continue with <a href="3.3.b_RNN_Ex_2.ipynb">exercise 2</a>. (Either locally or in Colab is fine.)</div>